In [ ]:
# importing libraries
import urllib.request, sys, time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
# news categories
category = ['India','Business', 'World', 'technology','movies','science','ipl-2022','happiness-quest']
pages_per_category = 10

In [ ]:
link_list = []
author_list = []
date_list = []
time_list =[]
category_list = [] 
content_list = []
headline_list = [] 
news_link_list=[]

In [ ]:
for i in category:

    for page in range(1,pages_per_category+1):
        url = 'https://www.indiatoday.in/{}?page={}'.format(i,page)
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")

        #news links
        news_link_list = soup.findAll('h2',class_='')
       

        # links
        for news_link in news_link_list:
          news_url = 'https://www.indiatoday.in' + news_link.find('a')['href']
          
          try:
            article_request = requests.get(news_url)
            article = BeautifulSoup(article_request.content, 'html.parser')
            content = article.find('div', attrs={'class':'node node-story view-mode-full'})


          #headline
            headline = content.find('h1',attrs={'itemprop':'headline'}).get_text()
            headline_list.append(headline)
            category_list.append(i)
            link_list.append(news_url)
          # news content
            news=''
            news_paras = article.find('div',attrs={'itemprop':"articleBody"}).find_all('p')
            for para in news_paras:
              news = news + para.get_text()              
            content_list.append(news)
            


          #author
            author_details = article.find('div',attrs={'class':"profile-detail"})
            try:
              author = author_details.find('a').get_text()
            except:
              author = 'author not available'
            author_list.append(author)

          # date and time
            date_time_details  = article.find('dt', attrs={'class':"update-data"})
            try:
              dateTime = date_time_details.get_text()
            except:
              dateTime = 'Date and time not available'

            date_list.append(dateTime)

          except:
            'URL Error'



In [ ]:
len(content_list),len(headline_list),len(author_list),len(link_list),len(category_list),len(date_list)

(429, 429, 429, 429, 429, 429)

In [ ]:
News_Corpus = pd.DataFrame({"Date": date_list, "Category": category_list,
                             "Headline": headline_list,"Entire_News": content_list, "Author": author_list, "News_Link": link_list})

In [ ]:
News_Corpus
date_new = [" ".join(i[1:4]) for i in News_Corpus.Date.str.split(" ")]
time_new = [" ".join(i[4:6]) for i in News_Corpus.Date.str.split(" ")]
del News_Corpus["Date"]
News_Corpus.insert(0,"Date",date_new)
News_Corpus.insert(1,"Time",time_new)
News_Corpus.insert(3,"Sub-Category",list(str(" ")*len(News_Corpus["Category"])))
News_Corpus["Entire_News"]=[i.split('ALSO')[0] for i in News_Corpus["Entire_News"]]

In [ ]:
#News_Corpus
News_Corpus_1=News_Corpus.drop_duplicates(subset=['News_Link']).drop_duplicates(subset=['Headline']).drop_duplicates(subset=['Entire_News'])
News_Corpus_1.Headline.value_counts()


'Enough stock to blow area in 10 minutes': TMC leader adds fire around Bengal's crude bomb recoveries     1
Pakistan SC overturns Imran Khan govt's actions, calls for no-trust vote on April 9                       1
US FBI claims it disrupted Russian military hackers                                                       1
West won't stop at destroying Russia; India will be next: Donetsk representative on Russia-Ukraine war    1
UN General Assembly to vote on Russia’s removal from Human Rights Council today                           1
                                                                                                         ..
Mukesh Ambani vs Gautam Adani: Who is winning see-saw battle of fortune?                                  1
Sensex drops 200 points, Nifty down to 17,992 amid inflation worries                                      1
Veranda Learning Solutions IPO share allotment: How to check application status                           1
HDFC merger: From 2nd larges

In [ ]:
News_Corpus_1.to_csv('IndiaToday.csv')

In [ ]:
from google.colab import files
files.download('IndiaToday.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>